In [2]:
!pip install imblearn


You are using pip version 19.0, however version 20.3.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [5]:
!pip install gensim

  Stored in directory: C:\Users\maryam\AppData\Local\pip\Cache\wheels\e5\2c\d5\d6460c7045050598b34945d98ecbc7423fafd4a846ce8b612c
Successfully built gensim


You are using pip version 19.0, however version 20.3.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [7]:
import pandas as pd
import numpy as np
from IPython.display import display
import matplotlib.pyplot as plt
from utilities import remove_empty_tweets

from imblearn.over_sampling import SMOTE
import gensim

from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_curve, precision_score, recall_score
from sklearn.metrics import classification_report

from sklearn.metrics import classification_report, f1_score


#text is already cleaned.
#assign cleaned data to these variables.
train_data_path = 'cleaned_data/cleaned_train_data_for_subtask1.csv'
test_data_path = 'cleaned_data/cleaned_test_data_for_subtask1.csv'
#read files.
train_data = pd.read_csv(train_data_path)
test_data = pd.read_csv(test_data_path)


print("Train set:"% train_data.columns, train_data.shape, len(train_data)) 
print("Test set:"% test_data.columns, test_data.shape, len(test_data)) 



Train set: (20974, 8) 20974
Test set: (4997, 8) 4997


In [8]:
#remove empty tweets.
remove_empty_tweets(train_data, "#2_tweet_clean_V1")
remove_empty_tweets(test_data, "#2_tweet_clean_V1")

,#1_tweetid,#2_tweet,#3_country_label,#2_tweet_clean_V0,#2_tweet_clean_V1,#2_tweet_clean_V2,#2_tweet_clean_V3,#classes_id
0,DEV_0,قولنا اون لاين لا يا علي اون لاين لا,Egypt,قولنا اون لاين لا يا علي اون لاين لا,قولنا اون لاين لا يا علي اون لاين لا,قولنا اون لاين يا اون لاين,قولنا اون لاين يا,0
1,DEV_1,ههههه بايخه ههههه URL …,Oman,ههههه بايخه ههههه,بايخه,بايخه,بايخه,6
2,DEV_2,ربنا يخليك يا دوك ولك المثل :D,Lebanon,ربنا يخليك يا دوك ولك المثل,ربنا يخليك يا دوك ولك المثل,ربنا يخليك يا دوك ولك المثل,ربنا يخليك يا دوك ولك المثل,8
3,DEV_3,#اوامر_ملكيه ياشباب اي واحد فيكم عنده شي يذكره...,Syria,اوامر ملكيه ياشباب اي واحد فيكم عنده شي يذكره ...,اوامر ملكيه ياشباب اي واحد فيكم عنده شي يذكره ...,اوامر ملكيه ياشباب فيكم عنده شي يذكره بالعساف ...,اوامر ملكيه ياشباب فيكم عنده شي يذكره بالعساف ...,5
4,DEV_4,شد عالخط حتى هيا اكويسه,Libya,شد عالخط حتى هيا اكويسه,شد عالخط حتى هيا اكويسه,شد عالخط اكويسه,شد عالخط اكويسه,13
...,...,...,...,...,...,...,...,...
4992,DEV_4995,و في الشتاء مستحيل يقب,Libya,و في الشتاء مستحيل يقب,في الشتاء مستحيل يقب,الشتاء مستحيل يقب,الشتاء مستحيل يقب,13
4993,DEV_4996,اكيد حبيبتي وانتِ في كيفك وكيف طيبة قلبك,Libya,اكيد حبيبتي وانتِ في كيفك وكيف طيبة قلبك,اكيد حبيبتي وانت في كيفك وكيف طيبه قلبك,اكيد حبيبتي وانت كيفك وكيف طيبه قلبك,اكيد حبيبتي وانت كيفك وكيف طيبه قلبك,13
4994,DEV_4997,يسعد صباح الناس رايقه,Jordan,يسعد صباح الناس رايقه,يسعد صباح الناس رايقه,يسعد الناس رايقه,يسعد الناس رايقه,18
4995,DEV_4998,فرحتهم فيهاا ياتقبرنييي USER .twitter.com/hUS...,Morocco,فرحتهم فيهاا ياتقبرنييي,فرحتهم فيها ياتقبرني,فرحتهم ياتقبرني,فرحتهم ياتقبرني,9


In [16]:
#prepare train and test data.
X_train = train_data['#2_tweet_clean_V1']
y_train = train_data['#classes_id']
X_test = test_data['#2_tweet_clean_V1']
y_test = test_data['#classes_id']
target_names = list(train_data["#3_country_label"].unique())

In [21]:
target_names

['Egypt',
 'Iraq',
 'Saudi_Arabia',
 'Mauritania',
 'Algeria',
 'Syria',
 'Oman',
 'Tunisia',
 'Lebanon',
 'Morocco',
 'Djibouti',
 'United_Arab_Emirates',
 'Kuwait',
 'Libya',
 'Bahrain',
 'Qatar',
 'Yemen',
 'Palestine',
 'Jordan',
 'Somalia',
 'Sudan']

In [24]:
# Parameters
sdg_params = dict(alpha=1e-5, penalty='l2', loss='log')
vectorizer_params = dict(ngram_range=(1, 2), min_df=5, max_df=0.8)

# Supervised Pipeline
pipeline = Pipeline([
    ('vect', CountVectorizer(**vectorizer_params)),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(**sdg_params)),
])

model = pipeline.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("Macro-averaged F1 score on test set: "
          "%0.3f" % f1_score(y_test, y_pred, average='macro'))
print(classification_report(y_test, y_pred,target_names=target_names))

Macro-averaged F1 score on test set: 0.171
                      precision    recall  f1-score   support

               Egypt       0.52      0.79      0.63      1041
                Iraq       0.36      0.50      0.42       663
        Saudi_Arabia       0.26      0.36      0.31       519
          Mauritania       0.69      0.21      0.32        53
             Algeria       0.38      0.37      0.37       430
               Syria       0.14      0.13      0.13       278
                Oman       0.21      0.23      0.22       355
             Tunisia       0.26      0.13      0.17       172
             Lebanon       0.14      0.05      0.08       157
             Morocco       0.16      0.07      0.10       207
            Djibouti       0.00      0.00      0.00        27
United_Arab_Emirates       0.17      0.09      0.12       157
              Kuwait       0.23      0.05      0.08       105
               Libya       0.35      0.28      0.31       314
             Bahrain      